In [2]:
import pandas as pd
import smtplib
import csv
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os
from datetime import datetime
import time
import logging

In [3]:
df = pd.read_csv("emails.csv")
df.head()

,EMAILS
0,info@cargen.com
1,elizabeth@charteredengineering.com
2,info@firstlenders.co.ke
3,amina@blackwoodhodge.com
4,sales@hydromaticsea.com


In [7]:
df.isna().sum()
display(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   EMAILS   186 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


None

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='bulk-email-sender.log',
)

,EMAILS
1,elizabeth@charteredengineering.com
66,ashleynyawira10@outlook.com
26,info@deafchildrensociety-kenya.org
15,usaidkea@usaid.gov
129,info@kemi.ac.ke
5,yusuf@kenyoswood.com
58,estherchiera1@gmail.com
143,customercare@gab.co.ke
81,ps@labour.go.ke
72,ps.rd@meac.go.ke


In [ ]:
class BulkEmailSender:
    def __init__(self, smtp_server, smtp_port, username, password, sender_email, sender_name=None):        
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.username = username
        self.password = password
        self.sender_email = sender_email
        self.sender_name = sender_name
        pass
    
    def connect(self):
        try:
            self.session = smtplib.SMTP(self.smtp_server, self.smtp_port)
            self.session.ehlo()
            self.session.starttls() # enable encryption
            self.session.ehlo()
            self.session.login(self.username, self.password)
            logging.info("Successfully connected to SMTP server")
        except Exception as e:
            logging.error(f"Failed to connect to SMTP server: {e}")
            return False
        return True
    
    def disconnect(self):
        if self.session:
            self.session.quit()
            self.session = None
            logging.info("Disconnected from SMTP server")
    
    def create_message(self, recipient_email, subject, html_content, text_content=None, attachment_paths=None, cc=None, bcc=None):
        msg = MIMEMultipart("alternative")
        msg["subject"] = subject
        msg["From"] = f"{self.sender_name} <{self.sender_email}>"
        msg["To"] = recipient_email
        
        if cc:
            msg["Cc"] = ", ".join(cc)
        if bcc:
            msg["Bcc"] = ", ".join(bcc)
        
        if text_content:
            msg.attach(MIMEText(text_content, "plain"))
        msg.attach(MIMEText(html_content, "html"))
        
        if attachment_paths:
            for file_path in attachment_paths:
                if os.path.exists(file_path):
                    with open(file_path, "rb") as f:
                        attachment = MIMEApplication(f.read(), Name=os.path.basename(file_path))
                        attachment["Content-Disposition"] = f"attachment; filename={os.path.basename(file_path)}"
                        msg.attach(attachment)
                else:
                    logging.warning(f"Attachment not found: {file_path}")
    
    def send_email(self, msg, recipient_email, cc=None, bcc=None):
        if not self.session:
            if not self.connect():
                return False
        
        recipients = [recipient_email]
        if cc:
            recipients.extend(cc)
        if bcc:
            recipients.extend(bcc)
        
        try:
            self.session.send_message(msg, self.sender_email, recipients)
            logging.info(f"Email sent to {recipient_email}")
            return True
        except Exception as e:
            logging.error(f"Failed to send email to {recipient_email}: {e}")
            return False
    
    def send_bulk_emails(self, csv_fiule, subject, html_template, text_template=None, attachment_paths=None, personalize=True, delay=2):
        pass